In [1]:
#include "../common.hpp"

using namespace std;
using namespace std::string_literals;

## RValue References

- _rvalue_ (right hand value) is an unnamed temporary
- `T&&` is used to denote a reference that can only bind to a temporary

```cpp
string str = "Hello"s;
string&& ref = str;
```
---
```
input_line_9:3:10: error: rvalue reference to type 'basic_string<...>' cannot bind to lvalue of type
      'basic_string<...>'
string&& ref = str;
         ^     ~~~
```

In [2]:
string&& ref = "Hello"s;

- A temporary value is safe to _consume_
- Useful to avoid copies
- A constructor taking the class type by rvalue reference is known as a _move constructor_
    - Similar to a copy constructor but it consumes it's argument

In [3]:
class movable {
    int* _some_data;
public:
    movable(movable&& x) noexcept : _some_data{x._some_data} // consume x
    { x._some_data = nullptr; } // leave x destructible
    
    //...
};

### Return Value Optimization
- _Return value optimization_ (RVO) avoids a copy (or move) on return by constructing the result in place
- RVO applies to _local named values_ and rvalue results
- Allowed optimziation since C++03, required by C++17

In [4]:
annotate f() {
    annotate x;
    return x;
}

annotate y = f();

annotate ctor


- Arguments to functions are in the caller scope
- RVO applies to passing an argument by value

In [5]:
void g(annotate x) { }
g(f());

annotate ctor
annotate dtor


- RVO does not apply to returning value argument
- C++11 defines returning a value argument as a _move_ operation

In [6]:
annotate h(annotate x) {
    return x;
}

annotate z = h(f());

annotate ctor
annotate move-ctor
annotate dtor


### Using RValue Refs and RVO to Avoid Copies
#### Make Classes Movable
- Provide a move constructor and move assignment operator
    - Compiler will provide them implicitely if
        - there are no user declared copy constructors, copy assignment operators, or destructors
        - all non-static data members and base classes are movable
    - To ensure you have them, declare them `= default`
    - Move constructor and move assignment should be declared `noexcept`
    - Post-condition of moved from object is _partially formed_ & can alias rhs for move assignment
    - Otherwise can assume no aliasing

Example:

In [7]:
class example_01 {
    int* _data;
public:
    explicit example_01(int x) : _data(new int(x)) { }
    ~example_01() { delete _data; }
    
    example_01(const example_01&) = delete;
    example_01& operator=(const example_01&) = delete;
    
    example_01(example_01&& x) noexcept : _data(x._data) { x._data = nullptr; }
    example_01& operator=(example_01&& x) noexcept {
        delete _data;
        _data = x._data;
        x._data = nullptr;
        return *this;
    }
    
    explicit operator int () { return *_data; }
};

In [8]:
class example_02 {
    unique_ptr<int> _data;
public:
    explicit example_02(int x) : _data(make_unique<int>(x)) { }
    // implicit dtor
    
    // implicit deleted copy-ctor and copy-assignment
    
    /*
        move-ctor and move-assignment would be provided by default, but declaring
        them ensures they are provided and correct.
    */
    example_02(example_02&&) noexcept = default;
    example_02& operator=(example_02&&) noexcept = default;
    
    explicit operator int () { return *_data; }
};

#### The Self Swap Problem
What is the post condition of:

In [9]:
string s = "Hello World!";
swap(s, s);

In [10]:
cout << s << endl;

Hello World!


`std::swap()` is defined as:
```cpp
template <class T>
void swap(T& a, T& b) {
    T tmp = move(a);
    a = move(b); // if a and b alias, then b has been moved from
    b = move(tmp);
}
```

```cpp
    example_01& operator=(example_01&& x) noexcept {
        delete _data;
        _data = x._data;
        x._data = nullptr;
        return *this;
    }
```

Is this okay?

In [12]:
example_01 e1(42);
swap(e1, e1);

In [13]:
cout << static_cast<int>(e1) << endl;

42


Testing